In [1]:
import glob 
import os 
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import numpy as np

import pandas as pd 
from torch.utils.data import Dataset, DataLoader
from torchvision import  utils

from PIL import Image

import p3_model2 as model 

#import p3_hw as hw
import sys
#import p3_utils2


from itertools import cycle
import torch.optim as optim
import p3_test2 as test
import time 

In [2]:
cwd = os.getcwd()
use_cuda = torch.cuda.is_available()
torch.manual_seed(123)
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
target_domain='real' # change here 
#domain_name=['sketch','quickdraw','infograph','real']
domain_name=['infograph']
#domain_name.remove(target_domain)

class_name=[file  for file in os.listdir(domain_name[0]) if file[-3:] !='csv' ]

csv_name_train={name: pd.read_csv(cwd+'/'+name+'/'+name+ '_train.csv',index_col=0) for name in domain_name}
csv_name_val={target_domain: pd.read_csv(cwd+'/'+target_domain+'/'+target_domain+ '_train.csv',index_col=0)}

train_csv=pd.concat(csv_name_train[key] for key in domain_name)
test_csv=csv_name_val[target_domain]

In [7]:

feature_size=2048
cnn_feature_extractor=torchvision.models.resnet50(pretrained=True) #resnet50 fc is for 1000 calsses
modules = list(cnn_feature_extractor.children())[:-1] # delete the last fc layer.
cnn_feature_extractor = nn.Sequential(*modules).to(device)

# set requires_grad to false
for param in cnn_feature_extractor.parameters():
    param.requires_grad = False
#encoder=model.SVHN_Extractor().cuda()
encoder=cnn_feature_extractor.cuda()

In [14]:
color_transform=transforms.Compose([
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
            ])
def extract_feature(frame):
    out_put_img=[]
    for idx,file in enumerate(frame.index):
        print(idx)
        image = Image.open(file)
        image=color_transform(image)
        print(image.size())
        encoder.eval()
        output = encoder(image)
        output=output.view(-1, feature_size)
        out_put_img.append(img)
    return out_put_img



img_train=extract_feature(train_csv)
img_val=extract_feature(test_csv)

0
torch.Size([3, 224, 224])


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 7, 7], but got 3-dimensional input of size [3, 224, 224] instead